In [4]:
import os
import ssl

import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
import spacy

!pip install spacy
!python -m spacy download en_core_web_dm

#import pyLDAvis
#import pyLDAvis.gensim
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords

from sklearn.datasets import fetch_20newsgroups


def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts, trigram_mod):
    [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts,
                  nlp,
                  allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):

    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# when run it in colab
!wget https://github.com/khider/INF549/tree/master/In-class%20exercises/Practicum-7/20news-bydate_py3.pkz --no-check-certificate


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
/usr/bin/python: No module named spacy
--2021-10-25 17:51:05--  https://github.com/khider/INF549/tree/master/In-class%20exercises/Practicum-7/20news-bydate_py3.pkz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/khider/INF549/blob/master/In-class%20exercises/Practicum-7/20news-bydate_py3.pkz [following]
--2021-10-25 17:51:05--  https://github.com/khider/INF549/blob/master/In-class%20exercises/Practicum-7/20news-bydate_py3.pkz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘20news-bydate_py3.pkz.2’

20news-bydate_py3.p

Import a corpus of documents and build a dictionary out of it

In [2]:
from gensim.test.utils import common_texts

print(common_texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [3]:
common_dictionary = corpora.Dictionary(common_texts)

print(list(common_dictionary.items()))

[(0, 'computer'), (1, 'human'), (2, 'interface'), (3, 'response'), (4, 'survey'), (5, 'system'), (6, 'time'), (7, 'user'), (8, 'eps'), (9, 'trees'), (10, 'graph'), (11, 'minors')]


In [4]:
# alternatively
print(common_dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


Build BOW representation of corpus of common texts

In [5]:
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
print(common_corpus)

[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


Show bow representation of corpus with actual words

In [6]:
print("* BOW corpus with actual words")
id_words = [[(common_dictionary[id], count) for (id, count) in line] for line in common_corpus]
print(id_words)

* BOW corpus with actual words
[[('computer', 1), ('human', 1), ('interface', 1)], [('computer', 1), ('response', 1), ('survey', 1), ('system', 1), ('time', 1), ('user', 1)], [('interface', 1), ('system', 1), ('user', 1), ('eps', 1)], [('human', 1), ('system', 2), ('eps', 1)], [('response', 1), ('time', 1), ('user', 1)], [('trees', 1)], [('trees', 1), ('graph', 1)], [('trees', 1), ('graph', 1), ('minors', 1)], [('survey', 1), ('graph', 1), ('minors', 1)]]


Build LDA model with 10 topics

In [7]:
lda = LdaModel(corpus = common_corpus, id2word = common_dictionary, num_topics=10)

Display learned topics

In [8]:
#for topic in lda.get_topics():
#    print(topic)

# display words numerically encoded
#for topic in lda.show_topics():
#    print(topic)

# diplay topics
pprint(lda.show_topics(num_words=len(common_dictionary)))

[(0,
  '0.187*"user" + 0.187*"system" + 0.098*"computer" + 0.098*"time" + '
  '0.098*"response" + 0.098*"survey" + 0.098*"eps" + 0.098*"interface" + '
  '0.009*"trees" + 0.009*"graph" + 0.009*"minors" + 0.009*"human"'),
 (1,
  '0.083*"graph" + 0.083*"trees" + 0.083*"system" + 0.083*"interface" + '
  '0.083*"computer" + 0.083*"user" + 0.083*"response" + 0.083*"minors" + '
  '0.083*"eps" + 0.083*"survey" + 0.083*"time" + 0.083*"human"'),
 (2,
  '0.292*"minors" + 0.292*"graph" + 0.153*"trees" + 0.153*"survey" + '
  '0.014*"system" + 0.014*"computer" + 0.014*"user" + 0.014*"human" + '
  '0.014*"eps" + 0.014*"interface" + 0.014*"response" + 0.014*"time"'),
 (3,
  '0.344*"trees" + 0.344*"graph" + 0.031*"system" + 0.031*"minors" + '
  '0.031*"time" + 0.031*"interface" + 0.031*"human" + 0.031*"computer" + '
  '0.031*"response" + 0.031*"user" + 0.031*"survey" + 0.031*"eps"'),
 (4,
  '0.262*"human" + 0.262*"interface" + 0.262*"computer" + 0.024*"trees" + '
  '0.024*"graph" + 0.024*"system" + 0.0

Consider new documents

In [9]:
new_documents = [
        ['computer', 'time', 'graph'],
        ['survey', 'response', 'eps'],
        ['human', 'system', 'computer']
    ]
new_corpus = [common_dictionary.doc2bow(text) for text in new_documents]

print(new_corpus)

[[(0, 1), (6, 1), (10, 1)], [(3, 1), (4, 1), (8, 1)], [(0, 1), (1, 1), (5, 1)]]


Get BOW representation of first new document

In [10]:
# get vector of document
bow_doc = common_dictionary.doc2bow(new_documents[0])

print(bow_doc)

[(0, 1), (6, 1), (10, 1)]


Embed document in space: express bow representation as convex combination of topics

In [11]:
print(">>>> topic prob distribution: lda[bow_doc]")
print(lda[bow_doc])

# or
# print(lda.get_document_topics(bow_doc))

>>>> topic prob distribution: lda[bow_doc]
[(0, 0.025007963), (1, 0.02500007), (2, 0.27498457), (3, 0.025011482), (4, 0.27499786), (5, 0.02500007), (6, 0.02500007), (7, 0.025000064), (8, 0.27499783), (9, 0.025000066)]


Update LDA model with new texts

In [12]:
lda.update(new_corpus)
pprint(lda.show_topics(num_words=len(common_dictionary)))

[(0,
  '0.213*"survey" + 0.213*"eps" + 0.213*"response" + 0.090*"system" + '
  '0.090*"user" + 0.049*"computer" + 0.049*"time" + 0.049*"interface" + '
  '0.008*"trees" + 0.008*"graph" + 0.008*"minors" + 0.008*"human"'),
 (1,
  '0.083*"graph" + 0.083*"trees" + 0.083*"system" + 0.083*"interface" + '
  '0.083*"computer" + 0.083*"user" + 0.083*"response" + 0.083*"minors" + '
  '0.083*"eps" + 0.083*"survey" + 0.083*"time" + 0.083*"human"'),
 (2,
  '0.500*"graph" + 0.177*"minors" + 0.097*"survey" + 0.097*"trees" + '
  '0.016*"computer" + 0.016*"time" + 0.016*"system" + 0.016*"user" + '
  '0.016*"human" + 0.016*"eps" + 0.016*"interface" + 0.016*"response"'),
 (3,
  '0.273*"graph" + 0.273*"trees" + 0.045*"system" + 0.045*"minors" + '
  '0.045*"time" + 0.045*"interface" + 0.045*"human" + 0.045*"computer" + '
  '0.045*"response" + 0.045*"user" + 0.045*"survey" + 0.045*"eps"'),
 (4,
  '0.582*"computer" + 0.228*"human" + 0.076*"interface" + 0.013*"time" + '
  '0.013*"graph" + 0.013*"system" + 0.01

Consider new corpus

In [14]:
    # to do only once
    # import nltk
    # nltk.download("stopwords")
    if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
            getattr(ssl, '_create_unverified_context', None)):
        ssl._create_default_https_context = ssl._create_unverified_context

    # define stop words
    stop_words = ['stop', 'the', 'to', 'and', 'a', 'in', 'it',
                  'is', 'I', 'that', 'had', 'on', 'for', 'were', 'was',
                  'from', 'subject', 're', 'edu', 'use']

    # nltk.download('stopwords')
    #stop_words = set(stopwords.words('english'))
    #stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

    # read corpus
    newsgroups_train = fetch_20newsgroups(subset='train', data_home="./")
    data = newsgroups_train.data
    
    print(data[:4])

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n", "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 1

Preprocess data

In [15]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

# it will print the data after prepared for stopwords
print(data[:4])

<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:1: Dep

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15 I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL ---- brought to you by your neighborhood Lerxst ---- ', 'From: (Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu A fair number of brave souls who upgraded th

Remove stop words and lemmatize texts

In [16]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
#trigram = gensim.models.Phrases(bigram[data], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

data_words_nostops = remove_stopwords(data, stop_words)
data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, nlp, allowed_postags=[
    'NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:4])  # it will print the lemmatized data.

[['where', 's', 'thing', 'car', 'nntp', 'post', 'host', 'park', 'line', 'wonder', 'out', 'there', 'enlighten', 'car', 'see', 'other', 'day', 'door', 'sport', 'car', 'look', 'be', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'where', 'car', 'make', 'history', 'info', 'have', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['si', 'poll', 'final', 'call', 'summary', 'final', 'call', 'si', 'clock', 'report', 'keyword', 'si', 'acceleration', 'clock', 'article', 'line', 'posting', 'host', 'fair', 'number', 'brave', 'soul', 'upgrade', 'si', 'clock', 'oscillator', 'have', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detail', 'experience', 'procedure', 'top', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'adapter', 'heat', 'sink', 'hour', 'usage', 'day', 'floppy', 'disk', 'functionality', 'floppy', 'b

Build LDA Model

In [17]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

print("* corpus in BOW format with actual words")
# print(corpus[:4])
# it will print the words with their frequencies.
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]])

print("\n* Build LDA model or read it from file")
in_file = False
if in_file:
    lda_model = LdaModel.load("big_lda_model")
else:
    lda_model = gensim.models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=id2word,
            num_topics=20,
            random_state=100,
            update_every=1,
            chunksize=100,
            passes=10,
            alpha='auto',
            per_word_topics=False
        )
    lda_model.save("big_lda_model")
        
doc_lda = lda_model[corpus]
pprint(lda_model.print_topics())

* corpus in BOW format with actual words
[[('addition', 1), ('be', 1), ('body', 1), ('bring', 1), ('bumper', 1), ('call', 1), ('car', 5), ('day', 1), ('door', 2), ('early', 1), ('engine', 1), ('enlighten', 1), ('front', 1), ('funky', 1), ('have', 1), ('history', 1), ('host', 1), ('info', 1), ('know', 1), ('late', 1), ('lerxst', 1), ('line', 1), ('look', 2), ('mail', 1), ('make', 1), ('model', 1), ('name', 1), ('neighborhood', 1), ('nntp', 1), ('other', 1), ('out', 1), ('park', 1), ('post', 1), ('production', 1), ('really', 1), ('rest', 1), ('s', 1), ('see', 1), ('separate', 1), ('small', 1), ('spec', 1), ('sport', 1), ('tellme', 1), ('thank', 1), ('there', 1), ('thing', 1), ('where', 2), ('wonder', 1), ('year', 1)], [('be', 2), ('call', 2), ('day', 2), ('have', 3), ('host', 1), ('line', 1), ('thank', 1), ('acceleration', 1), ('adapter', 1), ('add', 2), ('answer', 1), ('article', 1), ('attain', 1), ('base', 1), ('brave', 1), ('brief', 1), ('card', 1), ('clock', 4), ('cpu', 1), ('detail'

Retrieve the top 5 documents closest to a query document

In [26]:
from gensim.similarities import MatrixSimilarity
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile

index = MatrixSimilarity(doc_lda, num_features=len(id2word))
index.save("simIndex.index")

query_text = "my internet connection is slow I need to upgrade my service contract so that I can play games and stream high definition videos at the same time".lower().split()
vec_bow = id2word.doc2bow(query_text)
vec_lda = lda_model[vec_bow]

sims = index[vec_lda]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
#print(sims[:5])

# Print actual documents
for (id,p) in sims[:10]:
    print("({},{})".format(p,texts[id]))
    print()


(0.9966782927513123,['where', 'be', 'doubter', 'now', 'organization', 'cellar', 'public', 'access', 'system', 'line', 'write', 'article', 'make', 'cream', 'cheese', 'side', 'tomorrow', 'night', 'leafs', 'be', 'equal', 'then', 'again', 'maybe', 'put', 'mildly', 'watch', 'flyer', 'demolish', 'last', 'night', 'realize', 'matter', 'how', 'good', 'leafs', 'line', 'be', 'need', 'more', 'decent', 'line', 'go', 'far', 'playoff', 'course', 'healthy', 'darling', 'cellar', 'public', 'access', 'system', 'genie', 'darling', 'facthq', 'truth', 'technology'])

(0.9956613779067993,['phil', 'be', 'still', 'look', 'immaculate', 'line', 'technology', 'lady', 'gentleman', 'step', 'take', 'triumphant', 'trip', 'year', 'tonight', 'english', 'mulholland', 'era', 'tonight', 'game', 'nice', 'try', 'drabek', 'ninja'])

(0.994295597076416,['odometer', 'use', 'medical', 'center', 'radiology', 'line', 'nntp', 'posting', 'host', 'write', 's', 'scary', 's', 'way', 'set', 'arbitrary', 'mileage', 'figure', 'odometer',